In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/V1.4_Training.csv')
print(train_df.shape)
train_df.dropna(inplace=True)
print(train_df.shape)
train_df.head()
nonpol_samples = []
#sugg_samples = train_df.loc[train_df['Tag']==1]['Sentence']
#sugg_samples[0]
for i in range(len(train_df)):
    if(train_df.iloc[i]['Tag']==1):
        nonpol_samples.append(train_df.loc[i][1])

In [ ]:
nonpol_samples[0]

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
import string
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english')) 

In [ ]:
textp = ' '.join(nonpol_samples)
textp = textp.lower()
textp = textp.translate(textp.maketrans('','', string.punctuation))
textp = word_tokenize(textp)

In [ ]:
ngrams = lambda a, n: zip(*[a[i:] for i in range(n)])

In [ ]:
from collections import Counter
mc_np = Counter(ngrams(textp, 5)).most_common(4330)
mc_np

In [ ]:
mc_less_np = Counter(ngrams(textp, 5)).most_common(187)
mc_less_np

In [ ]:
def preprocess(df):
    inp = df.values
    for i in range(len(inp)):
        s = inp[i]
        sl = s.lower()
        sl = result = sl.translate(sl.maketrans('','', string.punctuation))
        tokens = word_tokenize(sl)
        #tokens = [w for w in tokens if not w in stop_words]
        inp[i] = tokens
    return inp

train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/V1.4_Training.csv')
train_df.dropna(inplace=True)
train_df = train_df[train_df['Tag']==1]['Sentence']
textall = preprocess(train_df)

In [ ]:
import numpy as np
en_model = {}
f = open('/content/drive/My Drive/data/glove.6B.50d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    en_model[word] = coefs
f.close()

print('Found %s word vectors.' % len(en_model))

In [ ]:
for t in textall:
    for w in t:
        if w in string.punctuation:
            t.remove(w)

In [ ]:
print(type(textall))
print(textall.shape)

In [ ]:
words=[]
keys=[]
i=1
print('starting')
for k, tsent in enumerate(textall):
    if (k%1000 == 0):
        print ('k', k, k/textall.size)
    for j, word in enumerate(tsent):
        #if (j%100 == 0):
            #print ('j', j)
        if word not in words:
            words.append(word)
            keys.append(i)
            i=i+1
print ('done!')

In [ ]:
len(words), len(keys)

In [ ]:
embed_len = len(words)
print (embed_len)

word2id = dict(zip(words, keys))
id2word = dict(zip(keys, words))
embeddings = np.zeros((embed_len+1,len(en_model['test'])))  # num_words * 50 (word vec len)
print('starting')
for i, w in enumerate(words):
    try:
        vec = en_model[w]
    except KeyError:
        pass
    embeddings[word2id[w]]=vec
print('done')

In [ ]:
def ngrams_lang_inp(arr):
    inp = []
    curr = []
    for s in arr:
        i=0
        while(i+4<len(s)):
            ngram = list(s[i:i+5])
            idngram = [word2id[w] for w in ngram]
            if i+5<len(s):
                nextword = word2id[s[i+5]]   
            else:
                break
            curr = [idngram, nextword]
            inp.append(curr)
            curr = []
            i=i+1
        
    return inp

In [ ]:
npol_text = []
nonpol_len = len(nonpol_samples)
print (nonpol_len)
print ('starting')
for i in range(nonpol_len):
    s = nonpol_samples[i]
    sl = s.lower()
    sl = result = sl.translate(sl.maketrans('','', string.punctuation))
    tokens = word_tokenize(sl)
    npol_text.append(tokens)
print ('finishing')

npol_lang_seq = ngrams_lang_inp(npol_text)
train_xp = np.array([s[0] for s in npol_lang_seq])
train_yp = np.array([s[1] for s in npol_lang_seq])

print('done')

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Embedding, Dense, Input, Bidirectional
from tensorflow.keras.models import Sequential

modelp = Sequential()
#model.add(Input(shape =(4,) , dtype='int32'))
modelp.add(Embedding(embeddings.shape[0], output_dim = 50,
                     weights = [embeddings], input_length=5, trainable = True))
modelp.add(Bidirectional(LSTM(256, activation = 'relu')))
modelp.add(Dense(1024, activation = 'relu'))
modelp.add(Dense(embeddings.shape[0], activation = 'softmax'))

modelp.compile(optimizer = 'Adam',
               loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
modelp.fit(train_xp, train_yp, batch_size = 32, epochs=20, verbose=True)

In [ ]:
modelp.save('/content/drive/My Drive/data/LMOTE_model_SM')

In [ ]:
nonpol_generated = []
for np in mc_np:
    ngram_gen = np[0] #('', '', '', '')
    idngram = []
    for p in ngram_gen:
        idngram.append(word2id[p])
    #[1, 2, 3, 4]
    ran_len = np.random.randint(70, 100)
    sentence = [id2word[i] for i in idngram]
    curr = idngram[:4]
    for l in range(ran_len):
        inp = np.array([curr, [2, 34, 5, 6]])
        p = np.argmax(modelp.predict(inp)[0])
        wordp = id2word[p]
        sentence.append(wordp)
        curr = np.delete(curr, 0)
        curr = np.append(curr, p)
    nonpol_generated.append(sentence)

In [ ]:
less_nonpol_generated = []

for lnp in mc_less_np:
    ngram_gen = lnp[0] #('', '', '', '')
    idngram = []
    for p in ngram_gen:
        idngram.append(word2id[p])
    #[1, 2, 3, 4]
    ran_len = np.random.randint(30, 40)
    sentence = [id2word[i] for i in idngram]
    curr = idngram[:4]
    for l in range(ran_len):
        inp = np.array([curr, [2, 34, 5, 6]])
        p = np.argmax(modelp.predict(inp)[0])
        wordp = id2word[p]
        sentence.append(wordp)
        curr = np.delete(curr, 0)
        curr = np.append(curr, p)
        
    npint = np.random.randint(0, len(mc_np))
    npseed = mc_pos[npint][0]
    npseed_idgram = [word2id[p] for p in npseed]
    ran_len_np = np.random.randint(20, 30)
    
    sentence_nonpol = [id2word[i] for i in npseed_idgram]
    currnp = npseed_idgram[:4]
    for l in range(ran_len_np):
        inp = np.array([currnp, [2, 34, 5, 6]])
        np = np.argmax(modelp.predict(inp)[0])
        wordnp = id2word[p]
        sentence_pos.append(wordnp)
        currnp = np.delete(currnp, 0)
        currnp = np.append(currnp, np)
    sentence = sentence + sentence_nonpol
    less_nonpol_generated.append(sentence)

In [ ]:
' '.join(less_nonpol_generated[100])

In [ ]:
all_gen_sentences = []
for s in less_nonpol_generated:
    all_gen_sentences.append(' '.join(s))

for s in nonpol_generated:
    all_gen_sentences.append(' '.join(s))
    
gen_df = pd.DataFrame(all_gen_sentences)    

In [ ]:
gen_df.to_csv('non_pol.csv')
!cp data.csv "drive/My Drive/data/"